In [41]:
import pandas as pd

In [42]:
from pathlib import Path

In [43]:
from concurrent.futures import ProcessPoolExecutor

In [44]:
project_dir = Path('.').resolve().parents[0]

In [45]:
data_dir = project_dir/'data'/'raw'

In [79]:
names = list(data_dir.iterdir())

In [86]:
names = sorted([n.name for n in names])

In [80]:
len(names)

296

In [88]:
names[:5]

['.gitkeep',
 '2021-10-03 10:04PM.tsv',
 '2021-10-03 10:06PM.tsv',
 '2021-10-03 10:21PM.tsv',
 '2021-10-03 11:12PM.tsv']

In [89]:
names[-5:]

['2021-10-23 08:38AM.tsv',
 '2021-10-23 09:46AM.tsv',
 '2021-10-23 10:53AM.tsv',
 '2021-10-23 12:01PM.tsv',
 '2021-10-23 12:47AM.tsv']

In [46]:
def read_csv(path):
    try:
        df = pd.read_csv(path, sep = '\t')
        df['parse_date'] = path.name.rstrip('.tsv')
        return df
    except:
        return None

In [47]:
with ProcessPoolExecutor() as pool:
    dfs = pool.map(read_csv, data_dir.iterdir())

In [48]:
dfs = list(dfs)

In [51]:
len(dfs)

296

In [91]:
df = pd.concat(dfs)

In [92]:
df.shape

(129753, 8)

In [93]:
df.head(1)

,provider,provider_id,name,price,original_price,hidden_price,URL,parse_date
0,bol,9200000119746345,Philips Sonicare ProtectiveClean 4300 HX6805/2...,49.99,NaN,49.99,https://bol.com/nl/nl/p/philips-sonicare-prote...,2021-10-23 10:53AM


In [94]:
def clean_numeric(s, to_type = float):
    if isinstance(s, str):
        return to_type(s.strip().strip('-'))
    else:
        return s

In [95]:
df['price'] = df['price'].apply(clean_numeric)

In [96]:
df.sort_values(by = 'price', inplace = True)

In [97]:
# df.drop_duplicates(subset = ['provider', 'provider_id'], keep = 'first', inplace = True)

In [98]:
df.head(1)

,provider,provider_id,name,price,original_price,hidden_price,URL,parse_date
226,bol,9300000007382505,OTRONIC® Relais Module 5v | Ideaal voor Arduin...,2.74,NaN,2.74,https://bol.com/nl/nl/p/otronic-relais-module-...,2021-10-23 09:46AM


In [99]:
df['parse_date'] = pd.to_datetime(df['parse_date'])

In [100]:
df.set_index('parse_date', inplace = True)

In [101]:
df.sort_index(inplace = True)

In [102]:
df.to_csv('combined_data.tsv', sep = '\t')

# EDA

In [70]:
groupby_cols = ['provider', 'provider_id']

In [90]:
df.groupby('provider')['name'].count().sort_values()

provider
mediamarkt    284
bcc           288
bol           575
Name: name, dtype: int64

In [61]:
df.describe()

,price,hidden_price
count,1145.000000,423.000000
mean,455.766969,275.654350
std,615.215406,602.809826
min,2.740000,2.740000
25%,42.830000,23.970000
50%,279.000000,47.950000
75%,669.150000,307.495000
max,8499.000000,7999.000000


In [62]:
df.shape

(1147, 7)

In [69]:
sum(df.reset_index().groupby(groupby_cols).count()['price']>1)

0

In [68]:
df.groupby(groupby_cols)[['price']].agg(['min', 'max', 'std'])

price             
                                   min      max std
provider   provider_id                             
bcc        000000000000048395   249.99   249.99 NaN
           000000000000079276    24.95    24.95 NaN
           000000000000219089    61.99    61.99 NaN
           000000000000219297   112.00   112.00 NaN
           000000000000219733   134.99   134.99 NaN
...                                ...      ...  ..
mediamarkt MMNL1707558         1279.00  1279.00 NaN
           MMNL1707687          499.00   499.00 NaN
           MMNL1707835          389.00   389.00 NaN
           MMNL1711411         2249.00  2249.00 NaN
           MMNL1711414         2979.00  2979.00 NaN

[1147 rows x 3 columns]

In [38]:
df[df.provider == 'mediamarkt']

,provider,provider_id,name,price,original_price,hidden_price,URL
parse_date,,,,,,,
2021-10-09 14:17:00,mediamarkt,MMNL1700318,LENOVO IdeaCentre 5 - i5-10400 8GB 512GB SSD,549.00,549.,NaN,https://mediamarkt.nl/nl/product/_lenovo-ideac...
2021-10-09 14:17:00,mediamarkt,MMNL1698756,MSI MAG META 5 3SI-441MYS,1139.00,1139.,NaN,https://mediamarkt.nl/nl/product/_msi-mag-meta...
2021-10-09 14:17:00,mediamarkt,MMNL1698956,HP OMEN 25L GT12-1410nd,1389.00,1389.,NaN,https://mediamarkt.nl/nl/product/_hp-omen-25l-...
2021-10-09 14:17:00,mediamarkt,MMNL1696650,HP OMEN 25L GT12-1420nd,1499.00,1499.,NaN,https://mediamarkt.nl/nl/product/_hp-omen-25l-...
2021-10-09 14:17:00,mediamarkt,MMNL1670320,"APPLE iMac 21.5"" - i5/8GB/1TB/FHD",999.00,1249.,NaN,https://mediamarkt.nl/nl/product/_apple-imac-2...
...,...,...,...,...,...,...,...
2021-10-09 17:22:00,mediamarkt,MMNL1698952,HP Pavilion TP01-2050nd,649.99,649.99,NaN,https://mediamarkt.nl/nl/product/_hp-pavilion-...
2021-10-09 17:22:00,mediamarkt,MMNL1698756,MSI MAG META 5 3SI-441MYS,1139.00,1139.,NaN,https://mediamarkt.nl/nl/product/_msi-mag-meta...
2021-10-09 17:22:00,mediamarkt,MMNL1701541,SCREENON GamePC V621817 - Ryzen 3 480GB,559.99,559.99,NaN,https://mediamarkt.nl/nl/product/_screenon-gam...


In [39]:
df[df.provider_id.isnull()]

,provider,provider_id,name,price,original_price,hidden_price,URL
parse_date,,,,,,,
